In [ ]:
!pip install feedparser # 설치 : RSS에서 xml태그별 정보를 추출(예: title, link, ...)
!pip install newspaper3k # 설치 : 인터넷신문기사분석(Article()을 사용하기 위한 목적) 파이썬 3버전이라 3k, 링크를 기사 들어가서 분석
!pip install konlpy # 설치 : 한국어 형태소 분석기 (명사만 추출할 목적)
!pip install soynlp # 설치 : 한국어 형태소 분석기 (명사만 추출할 목적)
!pip install lxml[html_clean] # 신규추가(2024년 가을)
import feedparser # !pip install feedparser
from newspaper import Article # !pip install newspaper3k
from konlpy.tag import Okt # !pip install konlpy
from collections import Counter # 본문에 몇번이나 명사가 나오는지 확인(TF구현용)
from bs4 import BeautifulSoup # 글에 존재할지 모르는 단어 추출

In [ ]:
# [단계1] 모든 RSS파일(xml형식)을 돌아다니면서 기사의 제목/link를 추출
# urls는 우리가 검색할 RSS파일들의 목록을 list로 만든 것
urls = ["https://rss.etnews.com/Section901.xml",
        "https://rss.etnews.com/Section902.xml",
        "https://rss.etnews.com/Section903.xml",
        "https://rss.etnews.com/Section904.xml"]
# 아래의 함수는 RSS목록의 list인 urls를 받아서 그 list에 존재하는
# 모든 RSS의 안에 있는 모든 기사들의 title과 link를 추출함
def crawl_rss(urls):
  array_rss = [] # 함수시작하는 시점에 빈 리스트를 만듦. 여기에 모든 기사의 title, link를 넣을 것임
  for index_url in urls: # urls 리스트안에 하나씩 xml을 방문 (4번 방문예정 : 901, 902, 903, 904)
    print("[Crawl RSS]", index_url) # 현재 어떤 xml파일을 방문중인지 표시
    parse_rss = feedparser.parse(index_url) # 현재 xml파일을 파싱 후, 결과를 parse_rss에 저장
    for p in parse_rss.entries: # parse_rss에 있는 모든 entries/기사를 검색하면서
      array_rss.append({'title':p.title,'link':p.link}) # append함수로 array_rss에 title과 link를 붙여넣음
  return array_rss
list_articles = crawl_rss(urls)
print(list_articles)

[Crawl RSS] https://rss.etnews.com/Section901.xml
[Crawl RSS] https://rss.etnews.com/Section902.xml
[Crawl RSS] https://rss.etnews.com/Section903.xml
[Crawl RSS] https://rss.etnews.com/Section904.xml
[{'title': '“머스크가 만든 온라인 커뮤니티, 5.8만명 모여 美 대선 가짜뉴스 공유”', 'link': 'https://www.etnews.com/20241105000196'}, {'title': "비트코인 '활황'에 빗썸·업비트 주문·차트 기능 대폭 개선", 'link': 'https://www.etnews.com/20241105000184'}, {'title': '더존비즈온, 3분기 영업익 201억 달성···성장세 이어간다', 'link': 'https://www.etnews.com/20241105000162'}, {'title': '한국, 베트남 노동자에 월급 제일 많이 주는 나라... “日보다 33% 높아”', 'link': 'https://www.etnews.com/20241105000129'}, {'title': '“러시아 진지서 발견된 북한군 시신…군모에 인공기”', 'link': 'https://www.etnews.com/20241105000128'}, {'title': "기아, 3년 만에 '더 뉴 스포티지' 출시…준중형 SUV 1위 굳힌다", 'link': 'https://www.etnews.com/20241105000114'}, {'title': '삼성, 3분기 태블릿 판매량 전년비 18.3%↑', 'link': 'https://www.etnews.com/20241105000044'}, {'title': "페루, 마추픽추 쓰레기에 '골머리'… “한국처럼 처리하라”", 'link': 'https://www.etnews.com/20241104000440'}, {'title': '한미,

In [ ]:
# [단계2] list에 존재하는 모든 기사드릐 link를 하나씩 돌아다니며 본문 title과 text 추출
# 아래의 함수는 하나의 url을 입력받아서, 그 링크를 타고 들어가, 그 안에 title과 text를 추출함.
def crawl_article(url, language='ko'):
  print("[Crawl Article]", url) # 현재 text를 추출할 기사의 url 출력
  article = Article(url, language=language) # Article을 사용하여 그 url을 입력하고, 언어옵션지정
  article.download() # 해당하는 url기사 다운로드
  article.parse() # 해당하는 url기사 분석
  return article.title, article.text # 해당하는 url기사 title과 text 출력

for article in list_articles: # 기존에 만든 list_articles에서 하나하나의 기사를 방문하면서
  _, text = crawl_article(article['link']) # 그 기사의 link를 crawl_article함수에 넣어 본문 추출
  article['text'] = text # 그 추출한 본문을 list_articles에 'text'라는 속성으로 새로 만들어 저장
print(list_articles[0])

[Crawl Article] https://www.etnews.com/20241105000196
[Crawl Article] https://www.etnews.com/20241105000184
[Crawl Article] https://www.etnews.com/20241105000162
[Crawl Article] https://www.etnews.com/20241105000129
[Crawl Article] https://www.etnews.com/20241105000128
[Crawl Article] https://www.etnews.com/20241105000114
[Crawl Article] https://www.etnews.com/20241105000044
[Crawl Article] https://www.etnews.com/20241104000440
[Crawl Article] https://www.etnews.com/20241105000013
[Crawl Article] https://www.etnews.com/20241104000364
[Crawl Article] https://www.etnews.com/20241104000315
[Crawl Article] https://www.etnews.com/20241105000004
[Crawl Article] https://www.etnews.com/20241104000445
[Crawl Article] https://www.etnews.com/20241104000434
[Crawl Article] https://www.etnews.com/20241104000386
[Crawl Article] https://www.etnews.com/20241104000316
[Crawl Article] https://www.etnews.com/20241104000432
[Crawl Article] https://www.etnews.com/20241104000414
[Crawl Article] https://www.

In [ ]:
# [단계3] 긁어온 본문 text에서 명사추출(키워드, 빈도수)
def get_keywords(text, nKeywords=10): # 키워드추출함수(빈도수고려: TF), 디폴트로 10개
  list_keywords = [] # 비어있는 키워드리스트를 먼저 만든다. 추후에 append를 써서 모든 기사에 대한 내용 추가
  spliter = Okt() # konlpy에 의해서 문장을 형태소별로 쪼개는 기능을 위해 spliter 생성
  nouns = spliter.nouns(text) # 입력받은 text에서 명사만 추출
  count = Counter(nouns) # 추출된 명사들의 출현빈도를 카운트
  for n, c in count.most_common(nKeywords): # 높은건 위, 낮은건 아래로 10번 연산
    item = {'keyword':n,'count':c} # {'keyword', 'count'}의 형식으로 저장
    list_keywords.append(item) # 위의 포멧으로 list_keywords에 저장
  return list_keywords


for article in list_articles: # 기존에 만든 list_articles에서 하나하나의 기사를 방문하면서
  keywords = get_keywords(article['text']) # 그 기사의 text를 get_keywords함수에 넣어 키워드/빈도 추출
  article['keywords'] = keywords # 그 추출한 키워드와 빈도수를 list_articles에 저장
print(list_articles[:5])

[{'title': '“머스크가 만든 온라인 커뮤니티, 5.8만명 모여 美 대선 가짜뉴스 공유”', 'link': 'https://www.etnews.com/20241105000196', 'text': "미국 공화당 대선후보인 도널드 트럼프를 공개 지지하는 일론 머스크 테슬라 최고경영자(CEO)가 5만 8000여 명의 회원이 있는 온라인 커뮤니티를 만들어 대통령 선거와 관련된 잘못된 정보를 퍼뜨리고 있다는 보도가 나왔다.\n\n\n\n머스크가 트럼프에게 자금을 지원하기 위해 구성한 '정치행동위원회(PAC)'는 지난달 엑스(X · 옛 트위터)에서 '선거 무결성 커뮤니티'(EIC)를 구성했다.\n\n\n\n2일(현지 시각) 미국 CBS 뉴스는 해당 커뮤니티 규모가 현재 5만 8000명까지 불어났고 매일 수백 개에 달하는 부정 선거 음모론을 퍼뜨리고 있다고 보도했다.\n\n\n\n일례로 투표 기계 오류가 발생해 트럼프가 아닌 민주당 대선후보 카멀라 해리스로 결과가 뒤집혔다는 주장이 담긴 가짜뉴스가 커뮤니티에서 확산됐으며, 한 선거 사무소에 투표 용지를 배달한 우체국 직원이 표를 가로챘다는 가짜뉴스가 퍼져 해당 직원의 신상을 캐는 '사이버 불링'이 이어지기도 했다.\n\n\n\n공개적으로 트럼프 후보를 지지하고 있는 머스크는 자신의 엑스 계정을 통해서도 선거와 관련한 게시물을 여럿 올리고 있다. CBS뉴스는 전문가들을 인용해 그가 올린 선거 보안에 관한 글 중 절반이 가짜뉴스 혹은 오해의 소지가 있는 주장이 담겼다고 했다.\n\n\n\n영국 싱크탱크 전략대화연구소(ISD)의 수석 연구원 맥스 리드는 “엑스 커뮤니티는 선거 조작 주장을 퍼뜨리고 싶은 사용자에게 '원스톱 숍' 역할을 한다”며 “선거 과정에 대한 다양한 거짓되고 검증되지 않은 주장이 엑스 커뮤니티에 통합되고 있다”고 지적했다.\n\n\n\n미국 CNN 방송은 비영리단체 디지털 증오 대응 센터(CCDH)를 인용해 머스크가 지난 7월 트럼프를 공개 지지한 이후 게시한 게시물의 조회수가 171억뷰 이상이며 이 

In [34]:
# [단계4] 검색어를 입력받아서 그 검색어를 가지고 있는 문서를 출력
query = input() # 쿼리를 입력 받음

# 아래의 함수는 쿼리를 입력받은 후에, 정해진 문서의 keywords의 리스트에서
# 쿼리가 그 keyword 중의 하나로 존재하는지 검색
def search_articles(query, list_keywords): # 쿼리가 키워드리스트에 있으면 빈도수 출력(없으면 0)
  nWords = 0 # 아래의 if문에 걸리지 않으면(즉, 쿼리가 키워드에 없으면) 0을 출력하기 위한 초기값
  for kw in list_keywords: # 키워드 리스트를 하나씩 kw로 검색
    if query == kw['keyword']: # 만약에 쿼리와 동일한 키워드가 존재한다면
      nWords = kw['count'] # 그렇다면, 그 키워드에 해당하는 count가 nWords가 됨
  return nWords # 결과적으로 쿼리의 출현횟수가 출력됨

for article in list_articles: # 기존에 만든 list_articles에서 하나의 기사를 방문하면서
  nQuery = search_articles(query, article['keywords']) # search_articles함수로 쿼리의 빈도수 추출
  if nQuery != 0: # 만약에 쿼리의 빈도수가 0이 아니면, 쿼리를 키워드로 가지고 있는 문서이므로, 관련 정보 출력
    print('[TF]', nQuery, '[Title]', article['title'])
    print('[URL]', article['link'])

개발
[TF] 4 [Title] 적자전환 엔씨소프트, '고강도 쇄신'으로 반등 모색
[URL] https://www.etnews.com/20241104000316
[TF] 4 [Title] 콘진원, 지스타 2024에서 게임인재원 졸업 프로젝트 전시
[URL] https://www.etnews.com/20241105000203
[TF] 6 [Title] [에듀플러스] 전북교육청 에듀테크 소프트랩, 12월까지 에듀테크 기업·교사 매칭 실증
[URL] https://www.etnews.com/20241105000190
[TF] 2 [Title] 태성, 유리기판 식각장비 개발
[URL] https://www.etnews.com/20241103000013
[TF] 4 [Title] 적자전환 엔씨소프트, '고강도 쇄신'으로 반등 모색
[URL] https://www.etnews.com/20241104000316
